In [ ]:
import pandas as pd
from datetime import datetime
import ast

In [ ]:
# This notebook is temporary. It is used to merge old dataframes with the new dataframe
# Main challenge is to get the older dataframe format in the new combined dataframe format
# This notebook will be deleted once every old dataframe is merged
# Around 1-11 the csv's are moved location

In [ ]:
date_csv = '2023-03-29'
old_df = pd.read_csv(f'../data/maersk_daily/csv_runs/connections_{date_csv }.csv',index_col=0)

In [ ]:
display(old_df)

In [ ]:
new_list = []

for index, row in old_df.iterrows():
    origin = row['Origin']
    destination =  row['Destination']
    departure_time =  row['LocalEstimatedDepartureTime']
    arrival_time = row['LocalEstimatedArrivalTime']
    est_transit_days = row['EstimatedTotalTransitTimeDays']
    ports = row['Ports']
    vessels = row['Vessels']
    dates = row['Dates']
    scraping_date = row['ScrapingDate']
    scraping_site = row['ScrapingSite']

    if index == 0:
        print()
        print('voor processen')
        print(origin)
        print(destination)
        print(departure_time)
        print(arrival_time)
        print(ports)
        print(vessels)
        print(dates)
        print('einde')
        print()

    origin_name = origin
    destination_name = destination

    # Remove the "datetime.datetime" prefix from the string representation
    dates = dates.replace("datetime.datetime", "")

    # Replace opening and closing brackets to create a valid list literal
    dates = "[" + dates[1:-1] + "]"

    # Use eval() to evaluate the string as a Python expression
    dates = eval(dates)

    # Convert the tuples to datetime objects
    datetime_objects = [datetime(*date) for date in dates]

    # Convert the datetime objects to dates in the desired format
    new_dates = [date.strftime("%Y-%m-%d %H:%M:%S") for date in datetime_objects]

    ports = ast.literal_eval(ports)

    if len(new_dates) < len(ports)*2-2:
        print()
        print('Warning at row ', index, 'Not enough dates for the amount of ports')
        print('dates', new_dates)
        print('ports', ports)
        print('appending empty dates')
        inserted_dates = 0
        for i in range(len(ports)-2):
            new_dates.insert(i+2+inserted_dates,'')
            inserted_dates += 1
        print(new_dates)


    vessels = ast.literal_eval(vessels)
    new_vessels_list = []
    for vessel in vessels:
        for i in range(len(vessel)):
            if vessel[i] == '-':
                vessel[i] = ''
            elif vessel[i] == 'unknown':
                vessel[i] = ''
        if len(vessel) < 6:
            for i in range(6-len(vessel)):
                vessel.append('')
        new_vessels_list.append({'vessel_name': vessel[0], 'imo': vessel[1], 'flag': vessel[3], 'build_year_ship': vessel[5],'service': vessel[2], 'callsign': vessel[4]})

    if len(new_vessels_list) < len(ports)-1:
        print()
        print('Warning at row ', index, 'Not enough vessels for the amount of ports')
        print('vessels', new_vessels_list)
        print('ports', ports)
        print('appending barge')
        for i in range(len(ports) - len(new_vessels_list) -1):
            new_vessels_list.append({'vessel_name': 'barge', 'imo': '', 'flag': '', 'build_year_ship': '','service': '', 'callsign': ''})
        print('new_vessels',new_vessels_list)
        print()

    legs = {}
    for leg in range(len(ports)-1):
        legs[f'{leg+1}'] = {'OriginName': ports[leg], 'DestinationName': ports[leg+1],'Vessel': new_vessels_list[leg],
                                                        'EstimatedDepartureTime': new_dates[leg*2], 'EstimatedArrivalTime': new_dates[leg*2+1]}

    est_transit_hours = ''
    co2= ''
    distance_meter = ''
    cutoffs = ''
    number_legs = ''

    new_list.append([scraping_date,scraping_site,origin,destination,origin_name,destination_name,departure_time,arrival_time,est_transit_days,est_transit_hours,co2,distance_meter,cutoffs,number_legs,legs])

In [ ]:
columns = ['ScrapingDate','ScrapingSite','Origin','Destination','OriginName','DestinationName','EstimatedDepartureTime','EstimatedArrivalTime','EstimatedTotalTransitTimeDays','EstimatedTotalTransitTimeHours','TotalCO2EmissionsKg','TotalDistanceMeters','CutOffs','NumberOfLegs','Legs']
new_df = pd.DataFrame(new_list, columns=columns)
display(new_df)

In [ ]:
combined_df = pd.read_csv(f'../data/maersk_daily/combined/combined2.csv',index_col=0)
new_combined_df = pd.concat([combined_df,new_df])
new_combined_df = new_combined_df.reset_index(drop=True)
new_combined_df = new_combined_df.applymap(lambda x: str(x) if isinstance(x, dict) else x)
new_combined_df = new_combined_df.drop_duplicates()
new_combined_df.head()

In [ ]:
new_combined_df.to_csv(f"../data/maersk_daily/combined/combined2.csv")